# Documentation

## Dataset

The dataset was generated using ChatGPT, following the format provided in the email.

I took some liberties myself. I think that the dataset will be better if it has a column for menu.

The dataset provided by ChatGPT is then saved as a CSV

# General Message

For this project, I decided to go with local models. I believe that cloud models can easily be implemented as the logic would still remain the same but will have superior performance compared to what I have.

I made this project in my laptop with the following specs

Processor: i5 12500H
RAM: 16GB
GPU RTX3060 Laptop GPU
VRAM: 6GB

To Setup

1. Make sure Ollama is installed. Ollama can be found at
2. Download Gemma3:1b model. The command is 
3. Download nomic: The command is


Gemma3 will be the LLM. Nomic will be the embedder

## Embedding

Below is the explanation of the embedding script

Importing necessary packages

Defining functions

Line chunk function chunks the csv to their respective rows, limiting the infomration to said row only

Get embed function takes in the chunk and run it through nomic. this should return a list of 768 numbers which is the embedding for said chunk. the function is built so that it takes in the chunk list and returns a list of embedding

store embed json stores the list of embeddings into a json file so that you won't have to perform the calculation over and over again